In [ ]:
import pandas as pd

In [ ]:
df_pthList=pd.read_csv('list_folder.txt',header=None)

In [40]:
a=[]
for i in df_pthList.loc[:,0]:
    if 'Energy_input_monthly.xls' in i:
        if 'P001' in i:
            #print(i.replace(' ','%20').replace('\\','/'))
            b=i.replace(' ','%20').replace('\\','/')
            a.append(b)
#print(a)
pth=pd.DataFrame(a)
pth=pth.loc[:,0].str.split(pat='/',expand=True)
pth.head()

,0,1,2,3,4,5,6,7,8,9,10
0,C:,inetpub,wwwroot,trnsysmodel,F00000000000001,1,P001,%209%20%20%20units,9%20units,Datalogger,Energy_input_monthly.xls
1,C:,inetpub,wwwroot,trnsysmodel,F00000000000001,1,P001,1%20unit,1%20unit,Datalogger,Energy_input_monthly.xls
2,C:,inetpub,wwwroot,trnsysmodel,F00000000000001,1,P001,1%20unit,b2,Datalogger,Energy_input_monthly.xls
3,C:,inetpub,wwwroot,trnsysmodel,F00000000000001,1,P001,1%20unit,b8,Datalogger,Energy_input_monthly.xls
4,C:,inetpub,wwwroot,trnsysmodel,F00000000000001,1,P001,1%20unit,bb,Datalogger,Energy_input_monthly.xls


In [42]:
df_getData=pd.DataFrame()
df_getData['factory']=pth.loc[:,4]
df_getData['system']=pth.loc[:,5]
df_getData['name']=pth.loc[:,8]
df_getData['link']="http://54.254.217.109/trnsysmodel"
for i in range(7):
    df_getData['link']=df_getData['link']+"/"+pth.loc[:,i+4]
df_getData.head()

,factory,system,name,link
0,F00000000000001,1,9%20units,http://54.254.217.109/trnsysmodel/F00000000000...
1,F00000000000001,1,1%20unit,http://54.254.217.109/trnsysmodel/F00000000000...
2,F00000000000001,1,b2,http://54.254.217.109/trnsysmodel/F00000000000...
3,F00000000000001,1,b8,http://54.254.217.109/trnsysmodel/F00000000000...
4,F00000000000001,1,bb,http://54.254.217.109/trnsysmodel/F00000000000...


In [46]:
df_temColData=pd.read_csv(df_getData.loc[0,'link'],sep='\t',skiprows=1,nrows=12).dropna(axis=1)
df_temColData.head()

,Period,Boiler1,Boiler2,Boiler3,Boiler4,Boiler5,Boiler6,Boiler7,Boiler8,Boiler9
0,January,3.353119e+08,3.353119e+08,3.353119e+08,3.353119e+08,3.353119e+08,3.353119e+08,3.353119e+08,3.353119e+08,6.706237e+08
1,February,3.028613e+08,3.028613e+08,3.028613e+08,3.028613e+08,3.028613e+08,3.028613e+08,3.028613e+08,3.028613e+08,6.057226e+08
2,March,3.353078e+08,3.353078e+08,3.353078e+08,3.353078e+08,3.353078e+08,3.353078e+08,3.353078e+08,3.353078e+08,6.706155e+08
3,April,3.244874e+08,3.244874e+08,3.244874e+08,3.244874e+08,3.244874e+08,3.244874e+08,3.244874e+08,3.244874e+08,6.489747e+08
4,May,3.352994e+08,3.352994e+08,3.352994e+08,3.352994e+08,3.352994e+08,3.352994e+08,3.352994e+08,3.352994e+08,6.705988e+08


In [73]:
def colToRow(df_temColData,factory,system,name):
    df_tod2=pd.DataFrame(columns=['no','Period','value'])
    month=df_temColData.loc[:,'Period']
    for k in range(df_temColData.shape[1]-1):
        temp=pd.DataFrame(month)
        temp['value']=df_temColData.iloc[:,k+1]
        temp['no']=k+1
        #print(temp.loc[:,['no','Period','value']])
        df_tod2=df_tod2.append(temp)
    df_tod2['factory']=factory
    df_tod2['system']=system
    df_tod2['name']=name
    return df_tod2.loc[:,['factory','system','name','no','Period','value']]
#colToRow(df_temColData,df_getData.loc[0,'factory'],df_getData.loc[0,'system'],df_getData.loc[0,'name'])


In [75]:

df=pd.DataFrame(columns=['factory','system','name','no','Period','value'])
for i in df_getData.index:
    try:
        df_temColData=pd.read_csv(df_getData.loc[i,'link'],sep='\t',skiprows=1,nrows=12)
        df_temColData=df_temColData.dropna(axis=1)
        df_tod=colToRow(df_temColData,df_getData.loc[i,'factory'],df_getData.loc[i,'system'],df_getData.loc[i,'name'])
        df=df.append(df_tod)
        print(df_temColData.shape,df.shape)
    except:
        print(i)

(12, 10) (108, 6)
(12, 2) (120, 6)
(12, 2) (132, 6)
(12, 2) (144, 6)
(12, 2) (156, 6)
(12, 2) (168, 6)
(12, 11) (288, 6)
(12, 11) (408, 6)
(12, 3) (432, 6)
(12, 4) (468, 6)
(12, 5) (516, 6)
(12, 6) (576, 6)
(12, 7) (648, 6)
(12, 8) (732, 6)
(12, 9) (828, 6)
15
(12, 2) (840, 6)
17
18
(12, 11) (960, 6)
(12, 10) (1068, 6)
(12, 4) (1104, 6)
(12, 2) (1116, 6)
(12, 6) (1176, 6)
(12, 2) (1188, 6)
(12, 2) (1200, 6)
(12, 2) (1212, 6)
(12, 2) (1224, 6)
(12, 2) (1236, 6)
(12, 6) (1296, 6)
(12, 2) (1308, 6)
(12, 3) (1332, 6)
(12, 6) (1392, 6)
(12, 6) (1452, 6)
(12, 2) (1464, 6)
(12, 3) (1488, 6)
(12, 3) (1512, 6)
(12, 3) (1536, 6)
(12, 4) (1572, 6)
(12, 4) (1608, 6)
(12, 5) (1656, 6)
(12, 5) (1704, 6)
(12, 5) (1752, 6)
(12, 2) (1764, 6)
(12, 2) (1776, 6)
(12, 6) (1836, 6)
(12, 4) (1872, 6)
(12, 3) (1896, 6)
(12, 3) (1920, 6)
